## Part 1

### Install BeautifulSoup and a parser (lxml)

In [1]:
pip install beautifulsoup4

     |████████████████████████████████| 102kB 15.6MB/s ta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install lxml

     |████████████████████████████████| 5.8MB 30.2MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


### Import libraries

In [3]:
import pandas as pd

In [4]:
from bs4 import BeautifulSoup
import urllib
import re
import requests

### Read url with xml parser

In [5]:
r = urllib.request.urlopen('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').read()

In [6]:
soup = BeautifulSoup(r, 'lxml')
type(soup)

bs4.BeautifulSoup

### Extract the table from the html

In [7]:
table = soup.find('table', {'class': 'wikitable sortable'})
print(table.prettify())

<table class="wikitable sortable">
 <tbody>
  <tr>
   <th>
    Postcode
   </th>
   <th>
    Borough
   </th>
   <th>
    Neighbourhood
   </th>
  </tr>
  <tr>
   <td>
    M1A
   </td>
   <td>
    Not assigned
   </td>
   <td>
    Not assigned
   </td>
  </tr>
  <tr>
   <td>
    M2A
   </td>
   <td>
    Not assigned
   </td>
   <td>
    Not assigned
   </td>
  </tr>
  <tr>
   <td>
    M3A
   </td>
   <td>
    <a href="/wiki/North_York" title="North York">
     North York
    </a>
   </td>
   <td>
    <a href="/wiki/Parkwoods" title="Parkwoods">
     Parkwoods
    </a>
   </td>
  </tr>
  <tr>
   <td>
    M4A
   </td>
   <td>
    <a href="/wiki/North_York" title="North York">
     North York
    </a>
   </td>
   <td>
    <a href="/wiki/Victoria_Village" title="Victoria Village">
     Victoria Village
    </a>
   </td>
  </tr>
  <tr>
   <td>
    M5A
   </td>
   <td>
    <a href="/wiki/Downtown_Toronto" title="Downtown Toronto">
     Downtown Toronto
    </a>
   </td>
   <td>
    <a href="

### Extract all the rows

In [8]:
output_rows = []
for t_row in table.findAll('tr'):
    columns = t_row.findAll('td')
    output_row = []
    for column in columns:
        output_row.append(column.text)
    output_rows.append(output_row)

In [9]:
output_rows

[[],
 ['M1A', 'Not assigned', 'Not assigned\n'],
 ['M2A', 'Not assigned', 'Not assigned\n'],
 ['M3A', 'North York', 'Parkwoods\n'],
 ['M4A', 'North York', 'Victoria Village\n'],
 ['M5A', 'Downtown Toronto', 'Harbourfront\n'],
 ['M5A', 'Downtown Toronto', 'Regent Park\n'],
 ['M6A', 'North York', 'Lawrence Heights\n'],
 ['M6A', 'North York', 'Lawrence Manor\n'],
 ['M7A', "Queen's Park", 'Not assigned\n'],
 ['M8A', 'Not assigned', 'Not assigned\n'],
 ['M9A', 'Etobicoke', 'Islington Avenue\n'],
 ['M1B', 'Scarborough', 'Rouge\n'],
 ['M1B', 'Scarborough', 'Malvern\n'],
 ['M2B', 'Not assigned', 'Not assigned\n'],
 ['M3B', 'North York', 'Don Mills North\n'],
 ['M4B', 'East York', 'Woodbine Gardens\n'],
 ['M4B', 'East York', 'Parkview Hill\n'],
 ['M5B', 'Downtown Toronto', 'Ryerson\n'],
 ['M5B', 'Downtown Toronto', 'Garden District\n'],
 ['M6B', 'North York', 'Glencairn\n'],
 ['M7B', 'Not assigned', 'Not assigned\n'],
 ['M8B', 'Not assigned', 'Not assigned\n'],
 ['M9B', 'Etobicoke', 'Cloverdale

### Find the column headings

In [10]:
headings = []
for th in table.findAll('th'):
    headings.append(th.text)
print(headings)

['Postcode', 'Borough', 'Neighbourhood\n']


### Create a dataframe and transpose the columns to rows

In [11]:
df = pd.DataFrame([output_rows]).transpose()
df.head(15)

,0
0,[]
1,"[M1A, Not assigned, Not assigned\n]"
2,"[M2A, Not assigned, Not assigned\n]"
3,"[M3A, North York, Parkwoods\n]"
4,"[M4A, North York, Victoria Village\n]"
5,"[M5A, Downtown Toronto, Harbourfront\n]"
6,"[M5A, Downtown Toronto, Regent Park\n]"
7,"[M6A, North York, Lawrence Heights\n]"
8,"[M6A, North York, Lawrence Manor\n]"
9,"[M7A, Queen's Park, Not assigned\n]"


### Split the column into multiple columns, sort the values and add column headers

In [12]:
can_df = df[0].apply(pd.Series)
can_df = can_df.sort_values([0])
can_df.columns = ['PostalCode', 'Borough', 'Neighbourhood']
can_df

,PostalCode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned\n
13,M1B,Scarborough,Malvern\n
12,M1B,Scarborough,Rouge\n
28,M1C,Scarborough,Highland Creek\n
29,M1C,Scarborough,Rouge Hill\n
...,...,...,...
246,M9W,Etobicoke,Northwest\n
259,M9X,Not assigned,Not assigned\n
275,M9Y,Not assigned,Not assigned\n
288,M9Z,Not assigned,Not assigned\n


### Remove \n

In [13]:
can_df['Neighbourhood'] = can_df['Neighbourhood'].str.replace('\n', '')
can_df.head()

,PostalCode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
13,M1B,Scarborough,Malvern
12,M1B,Scarborough,Rouge
28,M1C,Scarborough,Highland Creek
29,M1C,Scarborough,Rouge Hill


### Remove NA rows

In [14]:
can_df.dropna(inplace=True)
can_df

,PostalCode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
13,M1B,Scarborough,Malvern
12,M1B,Scarborough,Rouge
28,M1C,Scarborough,Highland Creek
29,M1C,Scarborough,Rouge Hill
...,...,...,...
230,M9V,Etobicoke,Beaumond Heights
246,M9W,Etobicoke,Northwest
259,M9X,Not assigned,Not assigned
275,M9Y,Not assigned,Not assigned


### Remove rows without a Borough

In [15]:
can_df = can_df[can_df.Borough != 'Not assigned']
can_df.reset_index(inplace=True)
can_df

,index,PostalCode,Borough,Neighbourhood
0,13,M1B,Scarborough,Malvern
1,12,M1B,Scarborough,Rouge
2,28,M1C,Scarborough,Highland Creek
3,29,M1C,Scarborough,Rouge Hill
4,30,M1C,Scarborough,Port Union
...,...,...,...,...
206,235,M9V,Etobicoke,South Steeles
207,229,M9V,Etobicoke,Albion Gardens
208,232,M9V,Etobicoke,Jamestown
209,230,M9V,Etobicoke,Beaumond Heights


### Show all rows unless specify a head

In [16]:
pd.set_option('display.max_rows', 999)
can_df.head()

,index,PostalCode,Borough,Neighbourhood
0,13,M1B,Scarborough,Malvern
1,12,M1B,Scarborough,Rouge
2,28,M1C,Scarborough,Highland Creek
3,29,M1C,Scarborough,Rouge Hill
4,30,M1C,Scarborough,Port Union


### Replace missing neighbourhood with borough

In [17]:
can_df2 = can_df.replace({'Not assigned': "Queen's Park"})
can_df2

,index,PostalCode,Borough,Neighbourhood
0,13,M1B,Scarborough,Malvern
1,12,M1B,Scarborough,Rouge
2,28,M1C,Scarborough,Highland Creek
3,29,M1C,Scarborough,Rouge Hill
4,30,M1C,Scarborough,Port Union
5,45,M1E,Scarborough,West Hill
6,44,M1E,Scarborough,Morningside
7,43,M1E,Scarborough,Guildwood
8,54,M1G,Scarborough,Woburn
9,63,M1H,Scarborough,Cedarbrae


### Remove redundant index

In [18]:
can_df2 = can_df2.drop('index', 1)
can_df2.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,Malvern
1,M1B,Scarborough,Rouge
2,M1C,Scarborough,Highland Creek
3,M1C,Scarborough,Rouge Hill
4,M1C,Scarborough,Port Union


In [19]:
can_df2.shape

(211, 3)

### Combine Neighborhoods in the same postcode and borough

In [20]:
can_df3 = can_df2.groupby(['PostalCode', 'Borough']).agg(', '.join)
can_df3.head(50)

,,Neighbourhood
PostalCode,Borough,
M1B,Scarborough,"Malvern, Rouge"
M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
M1E,Scarborough,"West Hill, Morningside, Guildwood"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae
M1J,Scarborough,Scarborough Village
M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
M1L,Scarborough,"Oakridge, Golden Mile, Clairlea"
M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"


### Reset index and you have the table

In [21]:
can_df3.reset_index(level=['PostalCode', 'Borough'], inplace=True)
can_df3.head(5)

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"West Hill, Morningside, Guildwood"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [22]:
can_df3.head(20)

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"West Hill, Morningside, Guildwood"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Oakridge, Golden Mile, Clairlea"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [23]:
can_df3.shape

(103, 3)

## End of part 1

## Part 2

### Read lat and long data

In [24]:
coordinates = pd.read_csv('Geospatial_Coordinates.csv')

In [25]:
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [26]:
coordinates.shape

(103, 3)

### Join tables

In [27]:
can_df4 = pd.merge(can_df3, coordinates, left_on='PostalCode', right_on='Postal Code')
can_df4.head()

,PostalCode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",M1C,43.784535,-79.160497
2,M1E,Scarborough,"West Hill, Morningside, Guildwood",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476


### Drop extra Postal Code column

In [28]:
can_df4 = can_df4.drop('Postal Code', 1)
can_df4.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"West Hill, Morningside, Guildwood",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


## End of part 2

## Part 3

### Import libraries

In [29]:
import numpy as np 
import json
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Libraries imported.


### Foursquare ID
__ID removed for github upload__

In [109]:
CLIENT_ID =  # your Foursquare ID
CLIENT_SECRET =  # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

### Map Toronto

In [35]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [37]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(can_df4['Latitude'], can_df4['Longitude'], can_df4['Borough'], can_df4['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Create function to get venues near each neighborhood

In [38]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Find venues

In [40]:
toronto_venues = getNearbyVenues(names=can_df4['Neighbourhood'],
                                   latitudes=can_df4['Latitude'],
                                   longitudes=can_df4['Longitude']
                                  )

Malvern, Rouge
Highland Creek, Rouge Hill, Port Union
West Hill, Morningside, Guildwood
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Oakridge, Golden Mile, Clairlea
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Wexford, Maryvale
Agincourt
Tam O'Shanter, Clarks Corners, Sullivan
Milliken, Steeles East, L'Amoreaux East, Agincourt North
L'Amoreaux West
Upper Rouge
Hillcrest Village
Oriole, Henry Farm, Fairview
Bayview Village
Silver Hills, York Mills
Willowdale, Newtonbrook
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Don Mills South, Flemingdon Park
Downsview North, Bathurst Manor, Wilson Heights
York University, Northwood Park
Downsview East, CFB Toronto
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Parkview Hill, Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West, 

### Get df of venues

In [41]:
print(toronto_venues.shape)
toronto_venues.head()

(2249, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Scarborough Historical Society,43.788755,-79.162438,History Museum
3,"West Hill, Morningside, Guildwood",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
4,"West Hill, Morningside, Guildwood",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


### Number of venues by neighborhood

In [56]:
Count_venues = toronto_venues.groupby('Neighborhood').count()
Count_venues.loc[:, 'Venue']

Neighborhood
Adelaide, King, Richmond                                                                                                                  100
Agincourt                                                                                                                                   4
Bayview Village                                                                                                                             4
Berczy Park                                                                                                                                56
Birch Cliff, Cliffside West                                                                                                                 4
Brockton, Exhibition Place, Parkdale Village                                                                                               24
Business Reply Mail Processing Centre 969 Eastern                                                                                      

### Enable df scrolling

In [61]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

### One hot encoding venues

In [69]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

toronto_onehot.head()


,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [73]:
toronto_onehot.shape

(2249, 270)

### Group venue categories into neighborhood

In [71]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,"Adelaide, King, Richmond",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000

In [72]:
toronto_grouped.shape

(99, 270)

### Top 10 venues by neighborhood

In [76]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [77]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Steakhouse,Bar,Asian Restaurant,Cosmetics Shop,Sushi Restaurant,Restaurant,Bakery,Hotel
1,Agincourt,Latin American Restaurant,Lounge,Skating Rink,Breakfast Spot,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
2,Bayview Village,Chinese Restaurant,Café,Bank,Japanese Restaurant,Women's Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
3,Berczy Park,Coffee Shop,Cheese Shop,Café,Farmers Market,Beer Bar,Cocktail Bar,Seafood Restaurant,Steakhouse,Bakery,Indian Restaurant
4,"Birch Cliff, Cliffside West",College Stadium,Café,Skating Rink,General Entertainment,Women's Store,Discount Store,Department Store,Dessert Shop,Dim Sum Restaurant,Diner


### Cluster with k-means

In [79]:
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 1, 3, 1, 1], dtype=int32)

### Add clusters to df

In [91]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
neighborhoods_venues_sorted.head(10)

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,"Adelaide, King, Richmond",Coffee Shop,Café,Steakhouse,Bar,Asian Restaurant,Cosmetics Shop,Sushi Restaurant,Restaurant,Bakery,Hotel
1,1,Agincourt,Latin American Restaurant,Lounge,Skating Rink,Breakfast Spot,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
2,1,Bayview Village,Chinese Restaurant,Café,Bank,Japanese Restaurant,Women's Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
3,1,Berczy Park,Coffee Shop,Cheese Shop,Café,Farmers Market,Beer Bar,Cocktail Bar,Seafood Restaurant,Steakhouse,Bakery,Indian Restaurant
4,1,"Birch Cliff, Cliffside West",College Stadium,Café,Skating Rink,General Entertainment,Women's Store,Discount Store,Department Store,Dessert Shop,Dim Sum Restaurant,Diner
5,1,"Brockton, Exhibition Place, Parkdale Village",Breakfast Spot,Café,Coffee Shop,Bakery,Grocery Store,Caribbean Restaurant,Stadium,Burrito Place,Sandwich Place,Restaurant
6,1,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Yoga Studio,Auto Workshop,Smoke Shop,Brewery,Spa,Burrito Place,Farmers Market,Fast Food Restaurant,Restaurant
7,3,Caledonia-Fairbanks,Park,Women's Store,Fast Food Restaurant,Market,Dumpling Restaurant,Drugstore,Eastern European Restaurant,Donut Shop,Dance Studio,Doner Restaurant
8,1,Canada Post Gateway Processing Centre,Hotel,Coffee Shop,Middle Eastern Restaurant,Fried Chicken Joint,Mediterranean Restaurant,Sandwich Place,Burrito Place,American Restaurant,Gym / Fitness Center,Dog Run
9,1,Cedarbrae,Bakery,Bank,Fried Chicken Joint,Hakka Restaurant,Athletics & Sports,Caribbean Restaurant,Thai Restaurant,Diner,Department Store,Dessert Shop


### Add back in lat & long

In [97]:
toronto_merged = pd.merge(neighborhoods_venues_sorted, can_df4, left_on='Neighborhood', right_on='Neighbourhood')

toronto_merged.head(10)

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,1,"Adelaide, King, Richmond",Coffee Shop,Café,Steakhouse,Bar,Asian Restaurant,Cosmetics Shop,Sushi Restaurant,Restaurant,Bakery,Hotel,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.650571,-79.384568
1,1,Agincourt,Latin American Restaurant,Lounge,Skating Rink,Breakfast Spot,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,M1S,Scarborough,Agincourt,43.794200,-79.262029
2,1,Bayview Village,Chinese Restaurant,Café,Bank,Japanese Restaurant,Women's Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,M2K,North York,Bayview Village,43.786947,-79.385975
3,1,Berczy Park,Coffee Shop,Cheese Shop,Café,Farmers Market,Beer Bar,Cocktail Bar,Seafood Restaurant,Steakhouse,Bakery,Indian Restaurant,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
4,1,"Birch Cliff, Cliffside West",College Stadium,Café,Skating Rink,General Entertainment,Women's Store,Discount Store,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848
5,1,"Brockton, Exhibition Place, Parkdale Village",Breakfast Spot,Café,Coffee Shop,Bakery,Grocery Store,Caribbean Restaurant,Stadium,Burrito Place,Sandwich Place,Restaurant,M6K,West Toronto,"Brockton, Exhibition Place, Parkdale Village",43.636847,-79.428191
6,1,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Yoga Studio,Auto Workshop,Smoke Shop,Brewery,Spa,Burrito Place,Farmers Market,Fast Food Restaurant,Restaurant,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558
7,3,Caledonia-Fairbanks,Park,Women's Store,Fast Food Restaurant,Market,Dumpling Restaurant,Drugstore,Eastern European Restaurant,Donut Shop,Dance Studio,Doner Restaurant,M6E,York,Caledonia-Fairbanks,43.689026,-79.453512
8,1,Canada Post Gateway Processing Centre,Hotel,Coffee Shop,Middle Eastern Restaurant,Fried Chicken Joint,Mediterranean Restaurant,Sandwich Place,Burrito Place,American Restaurant,Gym / Fitness Center,Dog Run,M7R,Mississauga,Canada Post Gateway Processing Centre,43.636966,-79.615819
9,1,Cedarbrae,Bakery,Bank,Fried Chicken Joint,Hakka Restaurant,Athletics & Sports,Caribbean Restaurant,Thai Restaurant,Diner,Department Store,Dessert Shop,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### Map Clusters

In [99]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ', Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Explore clusters

In [104]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2]

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,PostalCode,Borough,Neighbourhood,Latitude,Longitude
71,2,"Silver Hills, York Mills",Cafeteria,Women's Store,Discount Store,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dog Run,Curling Ice,M2L,North York,"Silver Hills, York Mills",43.75749,-79.374714


In [105]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4]

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,PostalCode,Borough,Neighbourhood,Latitude,Longitude
41,4,"Humberlea, Emery",Baseball Field,Women's Store,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Event Space,M9M,North York,"Humberlea, Emery",43.724766,-79.532242
46,4,"Kingsway Park South East, Old Mill South, Humb...",Construction & Landscaping,Baseball Field,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Women's Store,Deli / Bodega,M8Y,Etobicoke,"Kingsway Park South East, Old Mill South, Humb...",43.636258,-79.498509


In [106]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3]

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,PostalCode,Borough,Neighbourhood,Latitude,Longitude
7,3,Caledonia-Fairbanks,Park,Women's Store,Fast Food Restaurant,Market,Dumpling Restaurant,Drugstore,Eastern European Restaurant,Donut Shop,Dance Studio,Doner Restaurant,M6E,York,Caledonia-Fairbanks,43.689026,-79.453512
22,3,"Downsview East, CFB Toronto",Park,Snack Place,Airport,Discount Store,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Doner Restaurant,M3K,North York,"Downsview East, CFB Toronto",43.737473,-79.464763
25,3,Downsview West,Grocery Store,Convenience Store,Park,Bank,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,M3L,North York,Downsview West,43.739015,-79.506944
28,3,East Toronto,Park,Coffee Shop,Convenience Store,Women's Store,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,M4J,East York,East Toronto,43.685347,-79.338106
50,3,Lawrence Park,Park,Bus Line,Swim School,Diner,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Discount Store,Empanada Restaurant,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
54,3,"Martin Grove Gardens, Kingsview Village, St. P...",Park,Pizza Place,Bus Line,Discount Store,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dog Run,M9R,Etobicoke,"Martin Grove Gardens, Kingsview Village, St. P...",43.688905,-79.554724
55,3,"Milliken, Steeles East, L'Amoreaux East, Aginc...",Park,Playground,Women's Store,Discount Store,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,M1V,Scarborough,"Milliken, Steeles East, L'Amoreaux East, Aginc...",43.815252,-79.284577
67,3,Rosedale,Park,Playground,Trail,Dumpling Restaurant,Eastern European Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Cupcake Shop,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529
82,3,"The Kingsway, Old Mill North, Montgomery Road",Park,River,Women's Store,Diner,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Discount Store,M8X,Etobicoke,"The Kingsway, Old Mill North, Montgomery Road",43.653654,-79.506944
90,3,Weston,Park,Convenience Store,Women's Store,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doner Restaurant,M9N,York,Weston,43.706876,-79.518188


In [107]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0]

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,PostalCode,Borough,Neighbourhood,Latitude,Longitude
26,0,"Downsview, Upwood Park, North Park",Park,Construction & Landscaping,Bakery,Basketball Court,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Electronics Store,Deli / Bodega,M6L,North York,"Downsview, Upwood Park, North Park",43.713756,-79.490074
31,0,"Forest Hill North, Forest Hill West",Park,Jewelry Store,Sushi Restaurant,Trail,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Curling Ice,M5P,Central Toronto,"Forest Hill North, Forest Hill West",43.696948,-79.411307
37,0,"Highland Creek, Rouge Hill, Port Union",Bar,History Museum,Women's Store,Dance Studio,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
38,0,Hillcrest Village,Golf Course,Fast Food Restaurant,Athletics & Sports,Mediterranean Restaurant,Pool,Dog Run,Drugstore,Donut Shop,Doner Restaurant,Dance Studio,M2H,North York,Hillcrest Village,43.803762,-79.363452
42,0,Humewood-Cedarvale,Park,Field,Hockey Arena,Trail,Drugstore,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Curling Ice,M6C,York,Humewood-Cedarvale,43.693781,-79.428191
53,0,"Malvern, Rouge",Fast Food Restaurant,Women's Store,Dog Run,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doner Restaurant,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
61,0,"Oakridge, Golden Mile, Clairlea",Bus Line,Bakery,Soccer Field,Intersection,Park,Fast Food Restaurant,Metro Station,Dessert Shop,Dim Sum Restaurant,Diner,M1L,Scarborough,"Oakridge, Golden Mile, Clairlea",43.711112,-79.284577
65,0,Parkwoods,Park,Pool,Food & Drink Shop,Bus Stop,Donut Shop,Doner Restaurant,Drugstore,Dog Run,Curling Ice,Diner,M3A,North York,Parkwoods,43.753259,-79.329656
68,0,Roselawn,Music Venue,Garden,Women's Store,Curling Ice,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,M5N,Central Toronto,Roselawn,43.711695,-79.416936


In [108]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1]

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,1,"Adelaide, King, Richmond",Coffee Shop,Café,Steakhouse,Bar,Asian Restaurant,Cosmetics Shop,Sushi Restaurant,Restaurant,Bakery,Hotel,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.650571,-79.384568
1,1,Agincourt,Latin American Restaurant,Lounge,Skating Rink,Breakfast Spot,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,M1S,Scarborough,Agincourt,43.794200,-79.262029
2,1,Bayview Village,Chinese Restaurant,Café,Bank,Japanese Restaurant,Women's Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,M2K,North York,Bayview Village,43.786947,-79.385975
3,1,Berczy Park,Coffee Shop,Cheese Shop,Café,Farmers Market,Beer Bar,Cocktail Bar,Seafood Restaurant,Steakhouse,Bakery,Indian Restaurant,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
4,1,"Birch Cliff, Cliffside West",College Stadium,Café,Skating Rink,General Entertainment,Women's Store,Discount Store,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848
5,1,"Brockton, Exhibition Place, Parkdale Village",Breakfast Spot,Café,Coffee Shop,Bakery,Grocery Store,Caribbean Restaurant,Stadium,Burrito Place,Sandwich Place,Restaurant,M6K,West Toronto,"Brockton, Exhibition Place, Parkdale Village",43.636847,-79.428191
6,1,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Yoga Studio,Auto Workshop,Smoke Shop,Brewery,Spa,Burrito Place,Farmers Market,Fast Food Restaurant,Restaurant,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558
8,1,Canada Post Gateway Processing Centre,Hotel,Coffee Shop,Middle Eastern Restaurant,Fried Chicken Joint,Mediterranean Restaurant,Sandwich Place,Burrito Place,American Restaurant,Gym / Fitness Center,Dog Run,M7R,Mississauga,Canada Post Gateway Processing Centre,43.636966,-79.615819
9,1,Cedarbrae,Bakery,Bank,Fried Chicken Joint,Hakka Restaurant,Athletics & Sports,Caribbean Restaurant,Thai Restaurant,Diner,Department Store,Dessert Shop,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
10,1,Central Bay Street,Coffee Shop,Italian Restaurant,Ice Cream Shop,Sandwich Place,Burger Joint,Café,Salad Place,Japanese Restaurant,Gym / Fitness Center,Bubble Tea Shop,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383


Most neighborhoods seem to be in cluster 1  
Only 1 venue in cluster 2  
The two cluster 4 neighborhoods are not very near but seem to be quite similar   
Cluster 3 all seem to have parks around their top venues